In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from read_data import ReadClass

rc('text', usetex=True)
rc('font', family='serif')
fs = 16

experiment_name = "exRo10N3e06_LES"
figure_directory = "../figure/" + experiment_name
if not os.path.isdir(figure_directory):
    os.makedirs(figure_directory)

reader = ReadClass(experiment_name)
reader.read_geometry(0)

output_per_day = 86400 / reader.output_interval

def get_mean_spectrum_HV(nt, interval, variable, smooth=0):
    data, K_axis, M_axis = reader.get_spectrum_HV(nt, variable)
    if interval == 0:
        return data, K_axis, M_axis
    for it in range(nt+1, nt+interval):
        tmp, K_axis, M_axis = reader.get_spectrum_HV(it, variable)
        data = data + tmp
    data = data / interval
    if (smooth):
        b = np.ones(smooth) / smooth
        for j in range(np.shape(data)[1]):
            tmp = np.convolve(data[:, j], b, mode='same')
            data[:, j] = tmp
    return data, K_axis, M_axis

In [ ]:
import matplotlib.colors as mcolors

aspect = 1.0
log_on = True
plt.rcParams["font.size"] = 14


def ax_set(ax, K_axis, M_axis):
    ax.set_aspect(aspect)
    if log_on:
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlim(K_axis[1]/2, K_axis[-1])
        ax.set_ylim(M_axis[1]/2, M_axis[-1])
    else:
        ax.set_xlim(K_axis[0], K_axis[10])
        ax.set_yscale('log')
        ax.set_ylim(M_axis[1]/2, M_axis[-1])
    return


def get_colorbar_ticks(log_min, log_max):
    logarray = np.arange(log_min, log_max+1)
    linarray = 10.0**logarray
    ticks = np.concatenate([-linarray[::-1], np.zeros(1), linarray])
    tick_labels = [r'$-10^{{{exp}}}$'.format(exp=i) for i in logarray[::-1]]
    tick_labels.append('0')
    tick_labels.extend([r'$10^{{{exp}}}$'.format(exp=i) for i in logarray])
    return ticks, tick_labels


start_day = 8
interval_day = 2

nt = int(start_day * output_per_day)
interval = int(interval_day * output_per_day)
smooth = 3

PKE, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'PKE', smooth)
K, M = np.meshgrid(K_axis, M_axis)

PKE = PKE.transpose() / reader.output_interval * K * M
NKE, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'NKE', smooth)
NKE = NKE.transpose() / reader.output_interval * K * M
NPE, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'NPE', smooth)
NPE = NPE.transpose() / reader.output_interval * K * M
CKP, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'CKP', smooth)
CKP = CKP.transpose() / reader.output_interval * K * M
DKE, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'DKE', smooth)
DKE = DKE.transpose() / reader.output_interval * K * M
DPE, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'DPE', smooth)
DPE = DPE.transpose() / reader.output_interval * K * M

KFC, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'KFC', smooth)
KFC = KFC.transpose() / reader.output_interval * K * M
PFC, K_axis, M_axis = get_mean_spectrum_HV(nt, interval, 'PFC', smooth)
PFC = PFC.transpose() / reader.output_interval * K * M

log_min = -17
log_max = -13

ticks, tick_labels = get_colorbar_ticks(log_min, log_max)

vmax = ticks[-1]
vthresh = vmax * 10.e0**(log_min - log_max - 1)
loglevels = np.logspace(np.log10(vthresh), np.log10(vmax),
                        50, endpoint=True)
clevels = np.concatenate([-loglevels[::-1], loglevels])

norm = mcolors.SymLogNorm(vmin=-vmax, vmax=vmax,
                          linthresh=vthresh, linscale=vthresh)

fig = plt.figure(figsize=[8.6, 10])
fig.subplots_adjust(left=0.1, bottom=0.08, right=0.85,
                    top=0.98, wspace=0.06, hspace=0.2)

ax2 = fig.add_subplot(421)
cont = ax2.contourf(K_axis, M_axis, KFC,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax2, K_axis, M_axis)
ax2.set_ylabel(r'$m$ [rad/m]')
print(np.max(np.abs(- KFC)), np.sum(- KFC))

ax6 = fig.add_subplot(422)
cont = ax6.contourf(K_axis, M_axis, PFC,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax6, K_axis, M_axis)
print(np.max(np.abs(- PFC)), np.sum(- PFC))

ax1 = fig.add_subplot(423)
cont = ax1.contourf(K_axis, M_axis, PKE,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax1, K_axis, M_axis)
ax1.set_ylabel(r'$m$ [rad/m]')
print(np.max(np.abs(PKE)), np.sum(PKE))

ax5 = fig.add_subplot(424)
cont = ax5.contourf(K_axis, M_axis, CKP,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax5, K_axis, M_axis)
print(np.max(np.abs(CKP)), np.sum(CKP))

ax4 = fig.add_subplot(425)
cont = ax4.contourf(K_axis, M_axis, NKE,
                     cmap='RdBu_r', extend="both",
                     levels=clevels, norm=norm);
ax_set(ax4, K_axis, M_axis)
ax4.set_ylabel(r'$m$ [rad/m]')
print(np.max(np.abs(NKE)), np.sum(NKE))

ax8 = fig.add_subplot(426)
cont = ax8.contourf(K_axis, M_axis, NPE,
                     cmap='RdBu_r', extend="both",
                     levels=clevels, norm=norm);
ax_set(ax8, K_axis, M_axis)
print(np.max(np.abs(NPE)), np.sum(NPE))

ax3 = fig.add_subplot(427)
cont = ax3.contourf(K_axis, M_axis, - DKE,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax3, K_axis, M_axis)
ax3.set_xlabel(r'$k_H$ [rad/m]')
ax3.set_ylabel(r'$m$ [rad/m]')
print(np.max(np.abs(DKE)), np.sum(DKE))

ax7 = fig.add_subplot(428)
cont = ax7.contourf(K_axis, M_axis, - DPE,
                    cmap='RdBu_r', extend="both",
                    levels=clevels, norm=norm);
ax_set(ax7, K_axis, M_axis)
ax7.set_xlabel(r'$k_H$ [rad/m]')
print(np.max(np.abs(DPE)), np.sum(DPE))

cax2 = fig.add_axes((0.87, 0.35, 0.02, 0.3))
cbar = plt.colorbar(cont, orientation='vertical', cax=cax2, extend='both',
             ticks=ticks)
cbar.ax.set_yticklabels(tick_labels)
cbar.set_label(r'[W/kg]', labelpad=-40, y=1.18, rotation=0, fontsize=fs)

figure_name = figure_directory + '/' + 'budget_case1'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')